In [2]:
import brightway2 as bw

In [3]:
# Libary to import self-made libaries
import sys # https://stackoverflow.com/questions/4383571/importing-files-from-different-folder
sys.path.insert(1, r'C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care\Libaries')

In [4]:
import sensitivity as st
import Monte_Carlo as mc
import reload_lib as rl

In [5]:
# Importing self-made libaries
import standards as s
import life_cycle_assessment as lc
import LCA_plots as lp
import box_plot as bp
import reload_lib as rl


lib = [s, mc, lc, lp, bp, st]

In [6]:
# Path to where the code is stored
path = r'C:\Users\ruw\Desktop\RA\Single-use-vs-multi-use-in-health-care'
# Specifying the LCIA method
lcia_method = 'recipe'
rl.reload_lib(lib)

In [7]:
rl.reload_lib(lib)
flow_legend, database_name, file_name, sheet_name, save_dir, initialization, file_name_unique, db_type = lc.initilization(path, lcia_method)
# Setting up an empty dictionary with the flows as the key

project_name = initialization[0]
flows = initialization[2]
all_acts, eidb, eidb_db = lc.database_initialization(db_type, database_name, project_name)



size = len(flows)



The chosen project is SU_vs_MU and the chosen database is sterilization
sterilization_CONSQ already exist
Chosen flows:
H200 REC - CONSQ
H200 SU - CONSQ
H400 REC- CONSQ
H400 SU - CONSQ
alubox large + washer - CONSQ
alubox large + wipe - CONSQ
alubox small + washer - CONSQ
alubox small + wipe - CONSQ


In [10]:
# Obtaining all the product codes for the process'
func_unit = {}
procces_keys = {}

for act in all_acts:
    for proc in range(size):
        if act['name'] == flows[proc]:
            for sub in act.exchanges():
                if sub['type'] == 'technosphere':
                    func_unit[sub.input] = sub['amount']
                    procces_keys[sub.input] = sub.uncertainty
                    # if sub.uncertainty != {}:
                    # procces_keys[flows[proc]].update({sub.input : sub.uncertainty})
               

           



8

In [39]:
all_acts[0]
functional_unit = {all_acts[0] : 1}

In [52]:
import bw2data as bd
# Create a Monte Carlo LCA object
mc = bw.MonteCarloLCA(functional_unit)



background_processes = []
for sub in all_acts[0].exchanges():
    if sub['type'] == 'technosphere':
        background_processes.append(sub.input)


# Add uncertainty to the background processes
for process in background_processes:
    try:
        bd.utils.uncertainify(process, distribution='lognormal', bounds_factor=2)
    except AssertionError:
        print(f"{process} does not have any uncertainty")

'market for polypropylene, granulate' (kilogram, GLO, None) does not have any uncertainty
'sheet manufacturing - APOS' (kilogram, GLO, None) does not have any uncertainty
'packaging film production, low density polyethylene' (kilogram, RER, None) does not have any uncertainty
'market for corrugated board box' (kilogram, RER, None) does not have any uncertainty
'autoclave - APOS' (unit, GLO, None) does not have any uncertainty
'transport Plastic - APOS' (ton kilometer, GLO, None) does not have any uncertainty
'PP incineration no Energy Recovery - APOS' (kilogram, CH, None) does not have any uncertainty
'PE incineration no Energy Recovery - APOS' (kilogram, CH, None) does not have any uncertainty
'mixed electricity mix - APOS' (kilowatt hour, GLO, None) does not have any uncertainty
'mixed heating grid DK 23 - APOS' (megajoule, GLO, None) does not have any uncertainty
'treatment of waste paper to pulp, wet lap, totally chlorine free bleached' (kilogram, RoW, None) does not have any uncer

In [11]:
gwp_cat = lc.lcia_method(initialization[3])
gwp_cat = gwp_cat[1]
gwp_cat

Recipe is selected


('ReCiPe 2016 v1.03, midpoint (H) Runes edition',
 'climate change',
 'global warming potential (GWP1000)')

In [12]:
from copy import deepcopy as dc
import bw2calc as bc
import numpy as np


func_unit_uncert = dc(func_unit)

lcia_results = {key : [] for key in func_unit.keys()}
results = {key : {} for key in func_unit.keys()}

iterations = 1000
for itt in range(iterations):
    print(f"Iteration {itt + 1} of {iterations}")
    for act, val in func_unit.items():
        # if math.isnan(uncert['loc']) == False and math.isnan(uncert['scale']) == False and func_unit_uncert[key] != {}: 
        uncert = procces_keys[act]
        uncert_val = np.random.lognormal(uncert['scale'], np.exp(uncert['loc']))
        func_unit_uncert[act] = uncert_val
        print(f'Perfoming Monte Carlo  for {act}, val = {uncert_val}')
        # # Use the sampled data in the Monte Carlo LCA

        MC_lca = bc.MonteCarloLCA(func_unit_uncert, gwp_cat)
        MC_lca.lci()

        # Initialize cf_params if not already set
        if not hasattr(MC_lca, 'cf_params'):
            MC_lca.cf_params = MC_lca.load_lcia_data()
            # print('redo cf_params')

        # Rebuild the characterization matrix if it's not already initialized
        if not hasattr(MC_lca, 'characterization_matrix'):
            MC_lca.rebuild_characterization_matrix(MC_lca.method)
            # print('rebuild c matric')

        # Perform LCIA calculation directly
        MC_lca.lcia_calculation()

        results[act].update({f"itteration {itt}" : MC_lca.score})
        # print(act, func_unit_uncert[key][act])
    


Iteration 1 of 1000


KeyError: 'scale'

In [ ]:
import bw2calc as bc



In [13]:
import bw2data as bd
mc_functional_unit, mc_data_objs, _ = bd.prepare_lca_inputs(
    func_unit,
    method=gwp_cat,
)


AttributeError: module 'bw2data' has no attribute 'prepare_lca_inputs'

In [14]:

# 1. Define your foreground system (deterministic)
foreground_demand = {}

for act, val in func_unit.items():
    foreground_demand[(project_name, act)] = val

# 2. Specify the method (e.g., Global Warming Potential, IPCC 2013)

# 3. Create the LCA object with distributions enabled for the background
lca = bc.LCA(demand=func_unit, method=gwp_cat)
lca.lci()
lca.lcia()

# 4. Run the Monte Carlo simulation
iterations = 100  # Number of Monte Carlo runs
results = []

# for _ in range(iterations):
#     lca.redo_lcia()  # Resample the background uncertainty
#     results.append(lca.score)  # Store the impact score for each iteration

# # Results now contain 100 Monte Carlo samples
# print(results)


40416